# Silver: Transformations de la donnée

In [38]:
bronze_table_path = "abfss://79daad76-9b51-4f79-b1a3-5049702b5055@onelake.dfs.fabric.microsoft.com/c9815a16-8f35-44be-8356-61d5566edb18/Tables/dbo/wind_power"
df_bronze = spark.read.load(bronze_table_path)

display(df_bronze.summary())

StatementMeta(, c648129d-db98-425c-8f5e-8b7ca641230b, 40, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, c01cf482-b06c-4c42-9f5a-49db33a201bb)



### 🔢 Nettoyage numérique
   - Arrondir `wind_speed` à 2 décimales
   - Arrondir `energy_produced` à 2 décimales

In [39]:
import pyspark.sql.functions as sf

columns_to_round = ["wind_speed", "energy_produced"]

for column in columns_to_round:
    df_bronze = df_bronze.withColumn(column, sf.round(df_bronze[column], 2))

display(df_bronze.limit(5))

StatementMeta(, c648129d-db98-425c-8f5e-8b7ca641230b, 41, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, eeff6476-3eea-4cab-b8a4-0630745037de)

### 📅 Enrichissement temporel

- Extraire : `day`, `month`, `quarter`, `year`
- Extraire : `hour_of_day`, `minute_of_hour`, `second_of_minute`
- Corriger le format de time (remplacer "-" par ":")  



In [40]:
print(df_bronze.dtypes)

StatementMeta(, c648129d-db98-425c-8f5e-8b7ca641230b, 42, Finished, Available, Finished, False)

[('production_id', 'bigint'), ('date', 'timestamp'), ('time', 'string'), ('turbine_name', 'string'), ('capacity', 'bigint'), ('location_name', 'string'), ('latitude', 'double'), ('longitude', 'double'), ('region', 'string'), ('status', 'string'), ('responsible_department', 'string'), ('wind_speed', 'double'), ('wind_direction', 'string'), ('energy_produced', 'double')]


In [41]:
df_bronze = (df_bronze
    .withColumn("day", sf.dayofmonth(df_bronze.date))
    .withColumn("month", sf.month(df_bronze.date))
    .withColumn("quarter", sf.quarter(df_bronze.date))
    .withColumn("year", sf.year(df_bronze.date))
)

df_bronze = df_bronze.withColumn("time", sf.regexp_replace(df_bronze.time, "-", ":"))

splitted_time = sf.split(df_bronze.time, ":")
df_bronze = (df_bronze
    .withColumn("hour_of_day", splitted_time[0])
    .withColumn("minute_of_hour", splitted_time[1])
    .withColumn("second_of_minute", splitted_time[2])
)

display(df_bronze.orderBy(sf.rand()).limit(5))

StatementMeta(, c648129d-db98-425c-8f5e-8b7ca641230b, 43, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 9f2bd28c-ea3d-410c-8f62-8f8aad65ac9d)

### ✨ Calculs métier: la période de la journée
- `time_period` : Morning (5-11h), Afternoon (12-16h), Evening (17-20h), Night (autres)

In [42]:
df_bronze = df_bronze.withColumn("time_period", 
    sf.when((df_bronze.hour_of_day >= 5) & (df_bronze.hour_of_day <= 11), "Morning")
    .when((df_bronze.hour_of_day >= 12) & (df_bronze.hour_of_day <=16), "Afternoon")
    .when((df_bronze.hour_of_day >= 17) & (df_bronze.hour_of_day <= 20), "Evening")
    .otherwise("Night")
)

display(df_bronze.orderBy(sf.rand()).limit(5))

StatementMeta(, c648129d-db98-425c-8f5e-8b7ca641230b, 44, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 630451b9-5156-4f4d-b0e9-c8b352818671)

In [46]:
# Vérifier qu'il n'y a pas de valeurs nulles dans les colonnes critiques
print("=== 🔍 Vérification des valeurs nulles ===")
null_counts = df_bronze.select([
    sf.count(sf.when(sf.col(c).isNull(), c)).alias(c) 
    for c in ["wind_speed", "energy_produced", "day", "month", "year", "time_period"]
])
display(null_counts)

# Vérifier les valeurs uniques de time_period
print("\n=== 📊 Distribution des périodes de la journée ===")
display(df_bronze.groupBy("time_period").count().orderBy("count", ascending=False))

# Vérifier les plages de dates
print("\n=== 📅 Plage de dates ===")
display(df_bronze.select(
    sf.min("date").alias("Date minimale"),
    sf.max("date").alias("Date maximale")
))

# Statistiques descriptives
print("\n=== 📈 Statistiques sur les mesures ===")
display(df_bronze.select("wind_speed", "energy_produced").describe())

StatementMeta(, c648129d-db98-425c-8f5e-8b7ca641230b, 48, Finished, Available, Finished, False)

=== 🔍 Vérification des valeurs nulles ===


SynapseWidget(Synapse.DataFrame, 693ade48-1bb5-4fa7-b41e-025637df8213)


=== 📊 Distribution des périodes de la journée ===


SynapseWidget(Synapse.DataFrame, e35b9f2e-bd3d-48f9-b117-436733ee505f)


=== 📅 Plage de dates ===


SynapseWidget(Synapse.DataFrame, f6ddf988-92a8-4bee-b2dd-036517a71c9d)


=== 📈 Statistiques sur les mesures ===


SynapseWidget(Synapse.DataFrame, 93270624-bb69-42ec-aa6d-7ce5cef3c647)

In [47]:
silver_table_path = "abfss://79daad76-9b51-4f79-b1a3-5049702b5055@onelake.dfs.fabric.microsoft.com/f584c5bb-e8c1-42fe-ae55-db6de3ba417b/Tables/dbo/wind_power"
df_bronze.write.format("delta").mode("overwrite").save(silver_table_path)

StatementMeta(, c648129d-db98-425c-8f5e-8b7ca641230b, 49, Finished, Available, Finished, False)

## La même mais en sql (TODO)